# Задача: предсказание исходов профессиональных матчей по cs:go
## Этап: извлечение признаков 
### Описание этапа:
На данном этапе извлекаются признаки для турнирных игр по cs:go. Каждая игра имеет 2 команды, которые могут начать за 2 стороны(террористы и контр-террористы), игры состоят из раундов и каждый раунд может закончиться одним из исходов(взрыв бомбы, обезвреживание бомбы, вышедшее время раунда, полное уничтожение команды соперников). Игры проводятся в рамках различных турниров, каждый из которых имеет свой призовой фонд(сумма+валюта) и тир; игры имеют различный уровень важности: групповая стадия, плей-офф. В игре участвуют 10 игроков, по каждому из которых имеется регистрационная информация(год рождения, национальность и др) и статистическая информация.

* Игровые статистики:

|название|описание|
|---|---|
|adr|средний урон за раунд|
|assists|число ассистов|
|deaths|число смертей
|first_kills_diff|
|flash_assists|число световых гранат, способствовавших убийству|
|headshots|число убийств в голову|
|k_d_diff|отношение числа убиств к числу смертей|
|kast|%раундов,в которых игрок имел убийства, ассисты, выжил или был разменен|
|kills|число убийств
|rating|hltv рейтинг|

* hltv rating
Прошлая версия рейтинга состояла из трех компонентов — среднего количества фрагов в раунде, среднего количества раундов без смертей и дополнительной оценка, основанной на количестве раундов с двумя и более фрагами.
В финальной версии рейтинг 2.0 учитывает стороны, за которые выступал игрок. Обновленный параметр Impact Rating теперь включает среднее количество раундов с 2+ фрагами, клатчи и т.д. Появились два новых компонента — KAST Rating и Damage Rating. Первый отражает процент раундов без смертей, с фрагом, ассистом или разменом; второй — средний нанесенный урон в раунде.

### Признаки:

* принцип построения признаков: для игры агрегируется статистика по каждой команде во всех играх, состоявшихся до текущей на той же игровой карте                   
* список признаков:  

#### признаки для команды

    1. длина истории игр команды                  
    2. частоты бинаризованных длительностей игр
    3. частоты лет, месяцев, дней, дней недели, часов, десятков минут                   
    4. частоты исходов каждого раунда за каждую сторону с учетом результата раунда(взрыв бомбы, обезвреживание бомбыб ...)
    5. частоты категорий (тир серии, национальность, ...)
    6. суммарный призовой фонд в долларах
    7. частоты стадий турнира (группа, плей-офф, ...)
    8. ранжированные боевые статистики игроков в командах в пересчете на 1 раунд
    9. ранжированные личные боевые статистики игроков в пересчете на 1 раунд
    10. п.8 и п.9 для игр в рамках года, месяца, дня, дня недели, часа   
    
#### суммарное число выигранных раундов каждой командой в разрезе карт и сторон начала во всех играх, закончившихся до начала текущей      

### Целевой признак:

примеры целевого признака:   
    1. контр-террористы выиграют больше 10 раундов, а террористы больше3    
    2. всего будет сыграно меньше 25 раундов
    3. ...
    
### Дальнейшие действия:
    1. отбор признаков
    2. формирование целевых признаков

In [1]:
import pandas as pd
import numpy as np
import json, pickle, os, gc, re, time
from tqdm import tqdm, tqdm_notebook
from datetime import datetime
import dateutil.parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict

class CsgoFeatureGenerator():
    def __init__(self, PATH_TO_GAMES_OUT):
        self.PATH_TO_GAMES_OUT = PATH_TO_GAMES_OUT
    
    def _build_dataset(self):
        """
        на входе:
            путь к директории с обработанными играми из парсера
        на выходе:
            коллекция игр за все время в формате pd.DataFrame
        алгоритм:    
            1. загружаем все игры из директории с обработанными играми из парсера (каждая игра в формате json)
            2. делаем нормализацию каждого json и сохраняем
            3. трансформированные json преобразуем в pd.DataFrame        
        """        
        lstdr_in = os.listdir(self.PATH_TO_GAMES_OUT)
        order = np.argsort([int(i.split('.')[0]) for i in lstdr_in])
        lstdr_ordered_in = np.array(lstdr_in)[order]
        L_all_games = []
        for filename in tqdm(lstdr_ordered_in):
            pth = os.path.join(self.PATH_TO_GAMES_OUT, filename)
            with open(pth, 'r') as f:
                game_json = json.load(f)
            df_game = pd.json_normalize(game_json)
            d_game = df_game.iloc[0].to_dict()
            L_all_games.append(d_game)
        df = pd.DataFrame.from_records(L_all_games)
        df['begin_at'] = df['begin_at'].apply(lambda x: pd.to_datetime(x).tz_localize(None))
        df['end_at'] = df['end_at'].apply(lambda x: pd.to_datetime(x).tz_localize(None))
        df = df.sort_values('begin_at').reset_index(drop = True)
        return df
    
    def _get_game_history_4team(self, row, team1_id):
        """
        на входе:
            строка с игрой, идентификатор команды
        на выходе:
            коллекция игр команды до текущей игры в формате pd.DataFrame
        """        
        MASK_CurrentTeam_in_HistoryGamesTeams = self.df['teams'].apply(lambda x: team1_id in (x[0]['id'], x[1]['id']))
        MASK_CurrentGame_after_HistoryGames = self.df['begin_at']<row['begin_at']
        MASK=MASK_CurrentTeam_in_HistoryGamesTeams&MASK_CurrentGame_after_HistoryGames
        df_history4team= self.df[MASK]
        return df_history4team
    
    def _get_features4team(self, df_history_4team, team1_id):
        """
        на входе:
            история игр комадны, идентификатор команды
        на выходе:
            признаки для команды
        описание:
            1. мешки birthday (года месяц, день) игроков
            2. мешки hometown игроков 
            3. мешки nationality игрока 
            4. мешки nationality команд
            
            Для каждой карты(map_id):
            5. мешок бинаризованных длительностей игр (игры короче 16 минут исключаются как выбросы)
            6. мешки категорий (лига, серия, турнир, тир серии, размер призового фонда+валюта)
            7. средние ранжированые статистики ('kills', 'deaths', 'assists', 'flash_assists', 'headshots')
                в расчете на 1 раунд 
            8. средние ранжированые статистики ('adr', 'rating', 'k_d_diff', 'kast')
            9. мешок результатов каждого раунда в разрезе карт с учетом стороны начала для сторон ct, terrorists
        """       
        # словарь с признаками
        d_features = {}
        nrows = len(df_history_4team)
        for j, L_players in enumerate(df_history_4team['players']):
            for player in L_players:
                player_team_id = player['team']['id']
                if player_team_id == team1_id:            
                    player_reginfo = player['player']
                    player_bd= pd.to_datetime(player_reginfo['birthday'])
                    player_bd_year = player_bd.year
                    player_bd_month = player_bd.month
                    player_bd_day = player_bd.day
                    player_hometown = player_reginfo['hometown']
                    player_nationality = player_reginfo['nationality']
                    team_location = player['team']['location']                
                    try:
                        d_features['player.bd_year.{}.count'.format(player_bd_year)] += 1 / nrows
                    except:
                        d_features['player.bd_year.{}.count'.format(player_bd_year)] = 1 / nrows
                    try:
                        d_features['player.bd_month.{}.count'.format(player_bd_month)] += 1/ nrows
                    except:
                        d_features['player.bd_month.{}.count'.format(player_bd_month)] = 1/ nrows
                    try:
                        d_features['player.bd_day.{}.count'.format(player_bd_day)] += 1/ nrows
                    except:
                        d_features['player.bd_day.{}.count'.format(player_bd_day)] = 1/ nrows
                    try:
                        d_features['player.hometown.{}.count'.format(player_hometown)] += 1/ nrows
                    except:
                        d_features['player.hometown.{}.count'.format(player_hometown)] = 1/ nrows
                    try:
                        d_features['player.nationality.{}.count'.format(player_nationality)] += 1/ nrows
                    except:
                        d_features['player.nationality.{}.count'.format(player_nationality)] = 1/ nrows
                    try:
                        d_features['player.team_location.{}.count'.format(team_location)] += 1/ nrows
                    except:
                        d_features['player.team_location.{}.count'.format(team_location)] = 1/ nrows
        d_features = {k:round(v) for k,v in d_features.items()}                

        # делаем группировку по карте
        for history_map_id, subdf in df_history_4team.groupby('map.id'):

            # кол-во игр на карте
            key= 'map_id_{:.0f}.total_games'.format(history_map_id)
            d_features[key]= int(subdf.shape[0])

            # мешок бинаризованных длительностей игр (игры короче 16 минут исключаются как выбросы)
            length_minutes = subdf['length']//60
            length_minutes_round = round((subdf[length_minutes>=16]['length']//60)/10).astype(int).values
            for bin_length in length_minutes_round:
                key= 'map_id_{:.0f}.game_length_bin{}.count'.format(history_map_id, bin_length)
                try:
                    d_features[key] += 1
                except:
                    d_features[key] = 1

            s_league_id = subdf['match.league.id']
            s_serie_id = subdf['match.serie.id']
            s_tournament_id = subdf['match.tournament.id']
            s_serie_tier = subdf['match.serie.tier']
            s_tournament_name = subdf['match.tournament.name']
            s_tournament_prizepool = subdf['match.tournament.prizepool'] 

            # мешки категорий (лига, серия, турнир, тир серии, размер призового фонда+валюта)
            for s in [s_league_id, s_serie_id, s_tournament_id, s_serie_tier]:
                d_counts = s.value_counts().to_dict()
                for k, count in d_counts.items():
                    try:
                        k = round(float(k))
                    except:
                        pass
                    key = 'map_id_{:.0f}.'.format(history_map_id)+'_'.join(s.name.split('.')[1:]) + f'_{k}.count'
                    try:
                        d_features[key] += count
                    except:
                        d_features[key] = count            
            trnmt_pp_size = s_tournament_prizepool.replace({'DEFAULT':'0 United States Dollar'})\
                                                  .apply(lambda x: ''.join(re.findall(r'\d+', x.split(' ')[0]))).astype(int)
            trnmt_pp_currency= s_tournament_prizepool.apply(lambda x: str.lower(''.join(x.split(' ')[1:])))
            for pp_size, pp_currency in zip(trnmt_pp_size, trnmt_pp_currency):
                key = 'map_id_{:.0f}.'.format(history_map_id)+'tournament_prizepool_{}.sum'.format(pp_currency)
                pp_size, pp_currency
                try:
                    d_features[key] += pp_size
                except:            
                    d_features[key] = pp_size

            # средние ранжированые статистики в расчете на 1 раунд
            keys_stats_v1 = ['kills', 'deaths', 'assists', 'flash_assists', 'headshots']
            keys_stats_v2 =['adr', 'rating', 'k_d_diff', 'kast']
            DD_stats_v1 = defaultdict(list)
            DD_stats_v2 = defaultdict(list)
            for j, L_players in enumerate(subdf['players']):
                rounds_total_count = pd.DataFrame.from_records(subdf['rounds'].iloc[j])['round'].max()
                for player in L_players:
                    current_team_id= player['team']['id']
                    if current_team_id == team1_id:                
                        for k_stat in keys_stats_v1:
                            DD_stats_v1[k_stat].append(player[k_stat])
                        for k_stat in keys_stats_v2:
                            DD_stats_v2[k_stat].append(player[k_stat])
            D_stats_v1 = {k:sorted(v) for k, v in DD_stats_v1.items()}
            D_stats_v2 = {k:sorted(v) for k, v in DD_stats_v2.items()}
            for stat_key, stat_ranked_values in D_stats_v1.items():
                for rank_idx,value in enumerate(stat_ranked_values):
                    key = 'map_id_{:.0f}.{}.rank{}.mean.per_round'.format(history_map_id, stat_key, rank_idx+1)
                    d_features[key] = value / rounds_total_count  
            for stat_key, stat_ranked_values in D_stats_v1.items():
                for rank_idx,value in enumerate(stat_ranked_values):
                    key = 'map_id_{:.0f}.{}.rank{}.mean'.format(history_map_id, stat_key, rank_idx+1)
                    d_features[key] = value 
                    
            # агрегируем раунды в разрезе стороны начала, победителя, карты
            DD_rounds = defaultdict(list)
            for game in df_history_4team['rounds']:

                df_rounds= pd.DataFrame.from_records(game)
                r_min= df_rounds['round'].min()
                if r_min <=15:
                    subdf = df_rounds[df_rounds['round']== r_min]
                    start_ct = subdf['ct'].iloc[0]
                    start_t = subdf['terrorists'].iloc[0]
                else:
                    subdf = df_rounds[df_rounds['round']== r_min]
                    start_ct = subdf['terrorists'].iloc[0]
                    start_t = subdf['ct'].iloc[0]

                MASK_team1_start_ct = start_ct == team1_id
                MASK_team1_start_t = start_ct != team1_id   

                for r in game:
                    r_id = int(r['round'])
                    winner_id = int(r['winner_team'])

                    MASK_team1_win = winner_id == team1_id
                    MASK_team1_lose = winner_id != team1_id

                    DD_rounds['map_id_{:.0f}.start_ct.r{}.{}.win'\
                              .format(history_map_id, r_id, r['outcome'])].append(MASK_team1_start_ct & MASK_team1_win)
                    DD_rounds['map_id_{:.0f}.start_t.r{}.{}.win'\
                              .format(history_map_id, r_id, r['outcome'])].append(MASK_team1_start_t & MASK_team1_win)
                    DD_rounds['map_id_{:.0f}.start_ct.r{}.{}.lose'\
                              .format(history_map_id, r_id, r['outcome'])].append(MASK_team1_start_ct & MASK_team1_lose)
                    DD_rounds['map_id_{:.0f}.start_t.r{}.{}.lose'\
                              .format(history_map_id, r_id, r['outcome'])].append(MASK_team1_start_t & MASK_team1_lose)
            D_rounds = {k:np.mean(v) for k, v in DD_rounds.items()}
            d_features.update(D_rounds)
        return d_features    
    
    def _preapre_features(self, d_features4team1, d_features4team2):
        d_features4team1_c = {}
        for old_k, v in d_features4team1.items():
            new_k = 'TEAM1.'+old_k
            d_features4team1_c[new_k] = v
        del d_features4team1
        d_features4team2_c = {}
        for old_k, v in d_features4team2.items():
            new_k = 'TEAM1.'+old_k
            d_features4team2_c[new_k] = v
        del d_features4team2
        d_features4team1_c.update(d_features4team2_c)
        return d_features4team1_c
    
    def fit(self, PATH_TO_FEATURES_OUT, PATH_TO_TARGETS_OUT):        
        
        for _dir in [PATH_TO_FEATURES, PATH_TO_TARGETS]:
            if not(os.path.isdir(_dir)):
                os.mkdir(_dir)
            else:
                raise ValueError('directory already exists ...')
        
        self.df = self._build_dataset()        
        lstdr_out = os.listdir(PATH_TO_FEATURES_OUT)
        
        n_rows = len(self.df)
        # проходим по играм
        for i in tqdm(range(n_rows)):
            
            fn = '{}.pickle'.format(i)
            if fn not in lstdr_out:
                
                # текущая игра
                row = self.df.iloc[i]
                team1_id, team2_id, map_id = [int(t['id']) for t in row['teams']] + [int(row['map.id'])]

                # история игр команд
                df_history_4team1 = self._get_game_history_4team(row, team1_id)
                df_history_4team2 = self._get_game_history_4team(row, team2_id)            

                if (len(df_history_4team1)!=0) & (len(df_history_4team2)!=0):
                    mask_continue = True
                else:
                    mask_continue = False

                if mask_continue:
                    
                    # извлечение признаков 
                    d_features4team1 =  self._get_features4team(df_history_4team1, team1_id)  
                    d_features4team2 =  self._get_features4team(df_history_4team2, team2_id)  
                    d_features_row = self._preapre_features(d_features4team1, d_features4team2)
                    
                    # эеспорт признаков и результатов раундов в текущей игре
                    pth = os.path.join(PATH_TO_FEATURES_OUT, fn)
                    with open(pth, 'wb') as f:
                        pickle.dump(d_features_row, f)  
                    pth = os.path.join(PATH_TO_TARGETS_OUT, fn)
                    with open(pth, 'wb') as f:
                        pickle.dump(row['rounds'], f) 
                else:
                    pass
        
        return self

In [2]:
PATH_TO_GAMES_OUT = r'./games_out'
PATH_TO_FEATURES = r'./features_v1'
PATH_TO_TARGETS = r'./targets_v1'

In [ ]:
CsgoFeatGen = CsgoFeatureGenerator(PATH_TO_GAMES_OUT)
CsgoFeatGen.fit(PATH_TO_FEATURES, PATH_TO_TARGETS)

 16%|█▌        | 4493/27857 [00:26<02:21, 165.09it/s]

 65%|██████▍   | 18066/27857 [01:46<00:56, 171.87it/s]